In [153]:
import pickle

#data = [df,spray,(weather_st1, weather_st2)]

with open('cleanData.pickle', 'rb') as f:
    data = pickle.load(f)
    
df = data[0]

In [154]:
# Going to add this into the regular DF. Will become Deprecated Soon

df['Location'] = [(df.loc[idx,'Longitude'], df.loc[idx,'Latitude'])
                  for idx in df.index]

df.head(2)

,Date,Species,Trap,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,Yr,Mo,Day,Location
1379,2007-08-02,CULEX RESTUANS,T082,41.803423,-87.642984,8,3,0,2007,8,2,"(-87.642984, 41.803423)"
2704,2007-08-24,CULEX PIPIENS,T156,41.772846,-87.740029,9,3,0,2007,8,24,"(-87.740029, 41.772846)"


In [155]:
import shapefile
from scipy.spatial import cKDTree
import numpy as np
import pandas as pd

In [156]:
from os import listdir
from os.path import isfile, join

parkDir = './AddData/Parks/'
waterDir = './AddData/Water/'

parks = [f for f in listdir(parkDir) if isfile(join(parkDir,f)) if f.count('.csv') ==0]
water = [f for f in listdir(waterDir) if isfile(join(waterDir,f))if f.count('.csv') ==0]

parkShape = parkDir + parks[0].split('.')[0]
waterShape = waterDir + water[0].split('.')[0]

In [157]:
psf = shapefile.Reader(parkShape)
wsf = shapefile.Reader(waterShape)

parkSR = psf.shapeRecords()
waterSR = wsf.shapeRecords()

In [158]:
# Create cKDTree functions in Dict

# Key: Identifier (number for water, park name for park)
# Value: cKDTree function built on all the points associated with water/park feature

waterFinder={}
for i, s in enumerate(waterSR):
    waterFinder[i] = cKDTree(s.shape.points)

parkFinder = {}
for s in parkSR:
    parkFinder[s.record[4]] = cKDTree(s.shape.points)

In [159]:
# Create cKDTree functions in Dict

# Key: Identifier (number for water, park name for park)
# Value: cKDTree function built on all the points 
parkFinder={}
for k in pDict:   
    parkFinder[k] = cKDTree(pDict[k])

waterFinder = {}
for k in wDict:
    waterFinder[k] = cKDTree(wDict[k])

In [160]:
# Builds Distance Dictionaries:
# Key 1: Unique Location from DF
# Key 2: water/park ID
# Value: Distance from K1 to K2

uniqueLocs = df['Location'].unique()
parkDist = {}
waterDist = {}

for l in uniqueLocs:
    parkDist[l] = {}
    waterDist[l] = {}
    
    for k in parkFinder:
        parkDist[l][k]= parkFinder[k].query(l,1)[0]
    
    for k in waterFinder:
        waterDist[l][k] = waterFinder[k].query(l,1)[0]

In [164]:
parkDist[uniqueLocs[0]]


{'ABBOTT (ROBERT)': 0.08414113311150356,
 'ADA (SAWYER GARRETT)': 0.11449992380254197,
 'ADAMS (GEORGE & ADELE)': 0.11362753339904635,
 'ADAMS (JOHN C.)': 0.06344516633082568,
 'ADDAMS (JANE)': 0.06108759333746503,
 'ADDAMS (JANE) MEML.': 0.0933689651499472,
 'AIELLO (JOHN)': 0.17628810350119362,
 'ALGONQUIN': 0.14149275065424397,
 'ALMOND': 0.1233079927329591,
 'ALTGELD (JOHN)': 0.08560194150640492,
 'AMUNDSEN (ROALD)': 0.17318526985131735,
 'ANDERSON (FRED)': 0.05836930901222518,
 'ANDERSON (LOUIS)': 0.03128495629582209,
 'ANDERSONVILLE': 0.17556063779481582,
 'ARCADE': 0.11671515594812022,
 'ARCHER (WILLIAM BEATTY)': 0.09087680601537546,
 'ARMOUR (PHILIP) SQUARE': 0.030432681221087346,
 'ARMSTRONG (LILLIAN HARDIN)': 0.03302829008403601,
 'ARRIGO (VICTOR)': 0.06962148140962235,
 'ASHE (ARTHUR) BEACH': 0.09376408196327335,
 'ASHMORE': 0.19570054680750681,
 'ASPEN': 0.02298109079263853,
 'ASTER': 0.1636105722277511,
 'ATHLETIC FIELD': 0.16095994695817306,
 'AUBURN': 0.05020699899445052

In [165]:
waterDist[uniqueLocs[0]]

{0: 0.06865308305027652,
 1: 0.04939717299547288,
 2: 0.04976428213335435,
 3: 0.04786886065755066,
 4: 0.04230380994556612,
 5: 0.042262095609929995,
 6: 0.042125788462969486,
 7: 0.041822789315842046,
 8: 0.04164257150515251,
 9: 0.041607858670642615,
 10: 0.04142571900098145,
 11: 0.041203449197729,
 12: 0.0409314161436938,
 13: 0.030688197834117442,
 14: 0.05397213372995737,
 15: 0.05428194229001166,
 16: 0.05539836980121544,
 17: 0.05543798713086063,
 18: 0.05545381551598973,
 19: 0.05549961200449696,
 20: 0.11681674680977919,
 21: 0.18911522378744786,
 22: 0.055510590979319124,
 23: 0.05557667990387473,
 24: 0.05559763678631279,
 25: 0.055664755477256446,
 26: 0.0662900234902383,
 27: 0.06038404558798457,
 28: 0.06058286521744081,
 29: 0.06627332655492231,
 30: 0.066556732199503,
 31: 0.06532121666483177,
 32: 0.06534506214483198,
 33: 0.06757051733554004,
 34: 0.0677697913212864,
 35: 0.03054177515160165,
 36: 0.02546831329915466,
 37: 0.0831559517642503,
 38: 0.0755409283742358